# Text classification with Transformer

**Author:** [Apoorv Nandan](https://twitter.com/NandanApoorv)<br>
**Date created:** 2020/05/10<br>
**Last modified:** 2020/05/10<br>
**Description:** Implement a Transformer block as a Keras layer and use it for text classification.

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas
import numpy as np
from sklearn.model_selection import train_test_split


## Implement a Transformer block as a layer

In [2]:

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


## Implement embedding layer

Two seperate embedding layers, one for tokens, one for token index (positions).

In [3]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


## Load CSV file

In [4]:
def load_data(path="all_only_system_metric_20201120_integrated_9_features.csv"):
    df = pandas.read_csv(path,nrows=200)
    x = df.iloc[:,:-1].to_numpy(dtype='float16')
    y = df.iloc[:,-1].to_numpy(dtype='int32')
    x=[i.tolist() for i in x]
    x = np.array(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state = 120)
    return (x_train, y_train), (x_test, y_test)

## Prepare dataset

In [5]:
  # Only consider the top 20k words
maxlen = 9  # Only consider the first 200 pip install nbconvertwords of each movie review
# (x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
(x_train, y_train), (x_val, y_val) = load_data()
print( "Training sequences",len(x_train))
print( "Validation sequences",len(x_val))
# print("Training y_tranValue: ",y_train)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
# print("Training x_tranValue: ",x_train)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)
vocab_size = max(map(max, x_train))+1

Training sequences 140
Validation sequences 60


## Create classifier model using transformer layer

Transformer layer outputs one vector for each time step of our input sequence.
Here, we take the mean across all time steps and
use a feed forward network on top of it to classify text.

In [6]:

embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


## Train and Evaluate

In [7]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_val, y_val))

Epoch 1/10
5/5 [==============================] - 4s 369ms/step - loss: 0.6791 - accuracy: 0.5841 - val_loss: 0.6189 - val_accuracy: 0.7167
Epoch 2/10
5/5 [==============================] - 0s 38ms/step - loss: 0.5958 - accuracy: 0.6693 - val_loss: 0.5605 - val_accuracy: 0.7167
Epoch 3/10
5/5 [==============================] - 0s 50ms/step - loss: 0.4801 - accuracy: 0.7394 - val_loss: 0.5261 - val_accuracy: 0.7333
Epoch 4/10
5/5 [==============================] - 0s 43ms/step - loss: 0.4313 - accuracy: 0.7738 - val_loss: 0.4997 - val_accuracy: 0.7333
Epoch 5/10
5/5 [==============================] - 0s 35ms/step - loss: 0.3280 - accuracy: 0.8584 - val_loss: 0.4630 - val_accuracy: 0.8000
Epoch 6/10
5/5 [==============================] - 0s 33ms/step - loss: 0.2418 - accuracy: 0.9467 - val_loss: 0.4354 - val_accuracy: 0.8500
Epoch 7/10
5/5 [==============================] - 0s 36ms/step - loss: 0.1843 - accuracy: 0.9642 - val_loss: 0.4593 - val_accuracy: 0.8167
Epoch 8/10
5/5 [==========